In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
homestays_df = pd.read_csv("homestays.csv")
homestays_df.Address = homestays_df.Address.str.upper()
homestays_df.Address = homestays_df.Address.str.strip()
homestays_df.Address = homestays_df.Address.str.replace(",", "")
homestays_df.Address = homestays_df.Address.str.replace("STREET", "ST")
homestays_df.Address = homestays_df.Address.str.replace("ROAD", "RD")
homestays_df.Address = homestays_df.Address.str.replace("AVENUE", "AVE")
homestays_df.Address = homestays_df.Address.str.replace("LANE", "LN")
homestays_df.Address = homestays_df.Address.str.replace("DRIVE", "DR")
homestays_df.Address = homestays_df.Address.str.replace("PLACE", "PL")
homestays_df.Address = homestays_df.Address.str.replace("CIRCLE", "CIR")
homestays_df.Address = homestays_df.Address.str.replace("APT ", "#")

In [3]:
points_gdf = gpd.read_file("Master_Address_Points/Master_Address_Points.shp")
points_gdf.address = points_gdf.address.str.upper()
points_gdf.address = points_gdf.address.str.strip()
points_gdf.address = points_gdf.address.str.replace("STREET", "ST")
points_gdf.address = points_gdf.address.str.replace("ROAD", "RD")
points_gdf.address = points_gdf.address.str.replace("AVENUE", "AVE")
points_gdf.address = points_gdf.address.str.replace("LANE", "LN")
points_gdf.address = points_gdf.address.str.replace("DRIVE", "DR")
points_gdf.address = points_gdf.address.str.replace("PLACE", "PL")
points_gdf.address = points_gdf.address.str.replace("CIRCLE", "CIR")

In [4]:
# merge homestays and points datasets 
# note: had to manaully fix some addresses in homestays dataset that were misspelled or incorrect
homestays_points_gdf = pd.merge(homestays_df, points_gdf, left_on="Address", right_on="address")
gdf = gpd.GeoDataFrame(homestays_points_gdf, geometry="geometry", crs={'init': 'epsg:4326'})
# gdf = gdf.to_crs({'init': 'epsg:3857'}) # to meters
# gdf['geom_poly'] = gdf.geometry.buffer(20)
# gdf = gdf.set_geometry('geom_poly')
# gdf = gdf.to_crs({'init': 'epsg:4326'})
gdf = gdf[["Name", "Address", "approval date", "geometry"]]

In [5]:
airbnb_df = pd.read_csv("../scrapy-splash/airbnb/airbnb_cville.csv")
airbnb_df["geometry"] = [Point(xy) for xy in zip(airbnb_df.longitude, airbnb_df.latitude)]
airbnb_gdf = gpd.GeoDataFrame(airbnb_df, geometry="geometry", crs={'init': 'epsg:4326'})
airbnb_gdf = airbnb_gdf.to_crs({'init': 'epsg:3857'}) # to meters
airbnb_gdf['geom_poly'] = airbnb_gdf.geometry.buffer(50)
airbnb_gdf = airbnb_gdf.set_geometry('geom_poly')
airbnb_gdf = airbnb_gdf.to_crs({'init': 'epsg:4326'})
airbnb_gdf = airbnb_gdf[["accuracy", "location", "value", "geom_poly"]]

# with pd.option_context("display.max_columns", 50):
#     display(airbnb_gdf)

In [6]:
gdf.to_file("homestays.geojson", driver='GeoJSON')
airbnb_gdf.to_file("airbnb.geojson", driver='GeoJSON')

/Users/cwhalen/anaconda3/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [7]:
with pd.option_context("display.max_columns", 50):
    display(gpd.sjoin(gdf, airbnb_gdf, how="inner", op='within'))

,Name,Address,approval date,geometry,index_right,accuracy,location,value
10,"Williams, Julia",751 BELMONT AVE,11/4/16,POINT (-78.47765543494525 38.02492698492156),190,10.0,10.0,10.0
16,"Huerta, Thomas",1612 CAMBRIDGE CIR #A,3/21/17,POINT (-78.48767079284987 38.04935016782019),229,9.0,10.0,10.0
21,"McCune, Catherine",606 ALTAVISTA AVE #B,12/12/16,POINT (-78.48325503188701 38.0209025024897),100,10.0,10.0,10.0
21,"McCune, Catherine",606 ALTAVISTA AVE #B,12/12/16,POINT (-78.48325503188701 38.0209025024897),119,10.0,10.0,10.0
22,"Pearson, Kathleen",1614 OXFORD RD,12/15/16,POINT (-78.4881887606292 38.04760702021054),227,10.0,10.0,10.0
24,"Corcoran, Theodore",1855 WINSTON RD,8/14/16,POINT (-78.50058510013093 38.04687552210856),68,10.0,10.0,10.0
24,"Corcoran, Theodore",1855 WINSTON RD,8/14/16,POINT (-78.50058510013093 38.04687552210856),86,10.0,10.0,10.0
24,"Corcoran, Theodore",1855 WINSTON RD,8/14/16,POINT (-78.50058510013093 38.04687552210856),974,10.0,10.0,10.0
24,"Corcoran, Theodore",1855 WINSTON RD,8/14/16,POINT (-78.50058510013093 38.04687552210856),991,10.0,10.0,10.0
25,"Clayton, Signe",217 DOUGLAS AVE,11/15/16,POINT (-78.47447709185447 38.02591414213745),40,10.0,10.0,10.0


In [10]:
import numpy as np
tmp = gpd.sjoin(gdf, airbnb_gdf, how="inner", op='within')
len(np.unique(tmp.Address))

21